<a href="https://colab.research.google.com/github/harishpilla/Research/blob/Research/BreastCancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from sklearn.datasets import load_breast_cancer
data = load_breast_cancer()
x = data['data']
y = data['target']
print("shape of x: {}\nshape of y: {}".format(x.shape,y.shape))

shape of x: (569, 30)
shape of y: (569,)


In [ ]:
from sklearn.model_selection import train_test_split
 
from sklearn.metrics import accuracy_score, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 10)
from sklearn.svm import SVC
 
svc_model = SVC(C= .1, kernel='linear', gamma= 1)
svc_model.fit(X_train, y_train)
 
prediction = svc_model .predict(X_test)
print(svc_model.score(X_train, y_train))
print(svc_model.score(X_test, y_test))

0.9604395604395605
0.9473684210526315


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x = sc.fit_transform(x)

In [ ]:
from torch.utils.data import Dataset, DataLoader
class dataset(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    self.y = torch.tensor(y,dtype=torch.float32)
    self.length = self.x.shape[0]
 
  def __getitem__(self,idx):
    return self.x[idx],self.y[idx]
  def __len__(self):
    return self.length
trainset = dataset(x,y)
#DataLoader
trainloader = DataLoader(trainset,batch_size=64,shuffle=False)

In [ ]:
from torch import nn
from torch.nn import functional as F
class Feedforward(nn.Module):
  def __init__(self,input_size):
    super(Feedforward,self).__init__()
    self.fc1 = nn.Linear(input_size,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,1)
  def forward(self,x):
    x = torch.relu(self.fc1(x))
    x = torch.relu(self.fc2(x))
    x = torch.sigmoid(self.fc3(x))
    return x


class Feedforward_DLGN(nn.Module):
    def __init__(self, input_size):
        super().__init__()
        self.input_size = input_size
        self.fc1_npf = nn.Linear(input_size,32)
        self.fc1_npv = nn.Linear(input_size,32)
        self.fc2_npf = nn.Linear(32,64)
        self.fc2_npv = nn.Linear(32,64)
        self.fc3_npf = nn.Linear(64,1)
        self.fc3_npv = nn.Linear(64,1)


    def forward(self,x,x_npv):
        
        Sigmoid=torch.nn.Sigmoid()
        beta=10
        self.Gating_value=[]

        output11 = self.fc1_npf(x)
        Gate11=Sigmoid(beta*output11)

        output12=self.fc2_npf(output11)
        Gate12=Sigmoid(beta*output12)

        output13=self.fc3_npf(output12)
        Gate13=Sigmoid(beta*output13)
        

        ############################################## NPV Starts ##############################################################  

        output = self.fc1_npv(x_npv)
        output=output*Gate11

        output=self.fc2_npv(output)
        output=output*Gate12

        output=self.fc3_npv(output)
        output=output*Gate13   

        output = torch.sigmoid(output)
        # output = self.softmax(output)
        
        return output

In [ ]:
learning_rate = 0.01
epochs = 700
# Model , Optimizer, Loss
model = Feedforward(input_size=x.shape[1])
model_DLGN = Feedforward_DLGN(input_size=x.shape[1])
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)
optimizer_DLGN = torch.optim.SGD(model_DLGN.parameters(),lr=learning_rate)
loss_fn = nn.BCELoss()

In [ ]:
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    output = model(x_train)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model(torch.tensor(x,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  if i%10 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

epoch 0	loss : 0.6604838967323303	 accuracy : 0.8154657293497364
epoch 10	loss : 0.5672976970672607	 accuracy : 0.8347978910369068
epoch 20	loss : 0.4456852376461029	 accuracy : 0.9033391915641477
epoch 30	loss : 0.30781689286231995	 accuracy : 0.9384885764499121
epoch 40	loss : 0.21197590231895447	 accuracy : 0.9472759226713533
epoch 50	loss : 0.16032616794109344	 accuracy : 0.9543057996485061
epoch 60	loss : 0.13239306211471558	 accuracy : 0.9578207381370826
epoch 70	loss : 0.11570847034454346	 accuracy : 0.9630931458699473
epoch 80	loss : 0.10478395968675613	 accuracy : 0.968365553602812
epoch 90	loss : 0.09719635546207428	 accuracy : 0.9701230228471002
epoch 100	loss : 0.09152404218912125	 accuracy : 0.9736379613356766
epoch 110	loss : 0.08703997731208801	 accuracy : 0.9753954305799648
epoch 120	loss : 0.08336396515369415	 accuracy : 0.9771528998242531
epoch 130	loss : 0.08021079748868942	 accuracy : 0.9771528998242531
epoch 140	loss : 0.07741415500640869	 accuracy : 0.978910369068

In [ ]:
losses = []
accur = []
for i in range(epochs):
  for j,(x_train,y_train) in enumerate(trainloader):
    
    #calculate output
    All_ones=torch.ones(x_train.shape)
    All_ones_X=torch.ones(x.shape)
    output = model_DLGN(x_train,All_ones)
 
    #calculate loss
    loss = loss_fn(output,y_train.reshape(-1,1))
 
    #accuracy
    predicted = model_DLGN(torch.tensor(x,dtype=torch.float32),torch.tensor(All_ones_X,dtype=torch.float32))
    acc = (predicted.reshape(-1).detach().numpy().round() == y).mean()
    #backprop
    optimizer_DLGN.zero_grad()
    loss.backward()
    optimizer_DLGN.step()

  if i%10 == 0:
    losses.append(loss)
    accur.append(acc)
    print("epoch {}\tloss : {}\t accuracy : {}".format(i,loss,acc))

<ipython-input-10-91320d87ac85>:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  predicted = model_DLGN(torch.tensor(x,dtype=torch.float32),torch.tensor(All_ones_X,dtype=torch.float32))


epoch 0	loss : 0.6554350852966309	 accuracy : 0.6889279437609842
epoch 10	loss : 0.43950483202934265	 accuracy : 0.6379613356766256
epoch 20	loss : 0.2945592403411865	 accuracy : 0.6625659050966608
epoch 30	loss : 0.24835649132728577	 accuracy : 0.6818980667838312
epoch 40	loss : 0.23112568259239197	 accuracy : 0.7100175746924429
epoch 50	loss : 0.22279150784015656	 accuracy : 0.7328646748681898
epoch 60	loss : 0.2181202471256256	 accuracy : 0.7451669595782073
epoch 70	loss : 0.21528542041778564	 accuracy : 0.7574692442882249
epoch 80	loss : 0.21349851787090302	 accuracy : 0.7609841827768014
epoch 90	loss : 0.2123647928237915	 accuracy : 0.7644991212653779
epoch 100	loss : 0.21164442598819733	 accuracy : 0.7680140597539543
epoch 110	loss : 0.211139515042305	 accuracy : 0.7750439367311072
epoch 120	loss : 0.21067270636558533	 accuracy : 0.7785588752196837
epoch 130	loss : 0.21014462411403656	 accuracy : 0.7820738137082601
epoch 140	loss : 0.2095571756362915	 accuracy : 0.785588752196836